In [5]:
QPATH = "Quantlet/4-qode2desc"

import sys

IN_COLAB = "google.colab" in sys.modules

import os
import gc

if IN_COLAB:
    os.chdir(
        f"/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}"
    )

#%%capture
#%pip install protobuf==3.20.1
if IN_COLAB:
    %pip install transformers[torch]
    %pip install -q sentencepiece
    %pip install datasets==2.13.1
    %pip install evaluate
    %pip install rouge_score

In [6]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [7]:
%%capture
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

import torch
import torch, gc
import nltk

nltk.download("punkt")

import importlib
import analysis_modules

importlib.reload(analysis_modules)

In [8]:
for MODE in ['no_context', "domain"]:#
    analysis_config = {
        "DATE": "20231104",
        "MODE": MODE,
        "model_name": "CodeT5",
        "encoder_max_length": 512,
        "decoder_max_length": 75,
        "random_state": 42,
        "learning_rate": 5e-4,
        "epochs": 15,
        "train_batch": 4,
        "eval_batch": 4,
        "warmup_steps": 100,
        "weight_decay": 0.1,
        "logging_stes": 100,
        "save_total_lim": 1,
        "save_strategy": "steps",
        "label_smooting": 0.1,
        "predict_generate": True,
        "load_best_model_at_end": False,
        "evaluation_strategy": "epoch",
        "freeze": True,
    }
    if analysis_config["MODE"] == "domain":
        analysis_config["train_data_path"] = f"../../data/preprocessed/Quantlet/{analysis_config['DATE']}/no_context/"
    else:
        analysis_config["train_data_path"] = f"../../data/preprocessed/Quantlet/{analysis_config['DATE']}/{analysis_config['MODE']}/"

    analysis_config["train_data_name"] = f"full_train_dataset_{analysis_config['DATE']}_sample0.json"
    
    if analysis_config["MODE"] == "domain":
        analysis_config[
            "val_data_path"
        ] = f"../../data/preprocessed/Quantlet/{analysis_config['DATE']}/no_context/"
    else:
        analysis_config[
            "val_data_path"
        ] = f"../../data/preprocessed/Quantlet/{analysis_config['DATE']}/{analysis_config['MODE']}/"
    analysis_config[
        "val_data_name"
    ] = f"test_dataset_{analysis_config['DATE']}_sample0.json"

    analysis_config["analysis_name"] = analysis_modules.create_name(analysis_config)
    
    print(analysis_config["analysis_name"])

    if analysis_config["MODE"] == "domain":
        if analysis_config["model_name"] == "CodeT5":
            analysis_config[
                "model_name"
            ] = "../../data/pretrained/analysis_report_CodeT5-test-12-300-4-2023-09-26-v2/results/checkpoint-88488"
        if analysis_config["model_name"] == "CodeTrans":
            analysis_config[
                "model_name"
            ] = "../../data/pretrained/CodeTrans/results/checkpoint-12290"

    gc.collect()
    torch.cuda.empty_cache()

    trainer = analysis_modules.scs_analyze(**analysis_config)

    gc.collect()
    torch.cuda.empty_cache()

    logs = analysis_modules.parse_logs(trainer).drop_duplicates()

    ANALYSIS_FOLDER=f'reports/analysis_report_{analysis_config["analysis_name"]}'

    logs.to_csv(f'{ANALYSIS_FOLDER}/logs.csv', index=False)

    print('Analysis finished')

CodeT5_no_context_test_20231104
CodeT5_no_context_test_20231104
cuda
cuda


Found cached dataset json (/home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-6aaeab545afde721/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (/home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-7f5c8e4d974366b6/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

...

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      9.415        0.135        0.024        0.116           0.119   

   eval_bleu  eval_gen_len  
0      0.002        13.778  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,5.621200,4.708849,0.288700,0.107800,0.250100,0.255800,18.071900,0.037800,0.376100,0.505600,4093,8096
2,4.401000,4.350377,0.325600,0.143300,0.282300,0.290000,18.362500,0.058700,0.417400,0.533700,4321,8096
3,3.752500,4.168271,0.341000,0.166900,0.303200,0.309000,18.481200,0.077500,0.413100,0.530800,4297,8096
4,3.279200,4.094425,0.351000,0.184200,0.317400,0.324300,18.090600,0.087500,0.407300,0.526800,4265,8096
5,2.917400,4.122588,0.341800,0.188600,0.313400,0.318400,17.928100,0.086700,0.384200,0.511100,4138,8096
6,2.636900,4.071011,0.363100,0.204300,0.329900,0.337000,18.006200,0.095300,0.373300,0.503700,4078,8096
7,2.422800,4.043410,0.368600,0.207900,0.331400,0.337800,18.184400,0.101600,0.391000,0.515700,4175,8096
8,2.247300,4.036574,0.368700,0.215700,0.340500,0.346600,18.381200,0.108400,0.406700,0.526400,4262,8096
9,2.115900,4.050159,0.376100,0.221400,0.346800,0.353400,18.040600,0.109000,0.384800,0.511500,4141,8096
10,2.001300,4.057422,0.384200,0.228500,0.354400,0.361000,18.306200,0.118400,0.408900,0.527900,4274,8096


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      4.071        0.387        0.231        0.355           0.362   

   eval_bleu  eval_gen_len  
0      0.116        18.181  
__________
Original: Generates graphs and tables to evaluate prediction performance of naive predictor, LASSO, and LSTM models.


Summary before Tuning: load Error Measures for Applications in Michael K


Summary after Tuning: The Quantlet is used to predict the return of the predictor model to obtain error measures and to compute the predictions of energy consumption values and predicted error measures.


__________



__________
Original: Generates plots of total over-/underestimation errors of naive, LASSO, and LSTM models for multiple energy consumer and prosumer data sets.


Summary before Tuning: The Path, Id, and Return: Michael K


Summary after Tuning: Plots exemplary the density function of one household's raw energy consumption values and the density function its log energy con

In [ ]:
from huggingface_hub import scan_cache_dir

In [ ]:
cache = scan_cache_dir()

In [ ]:
cache